In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense

import pandas as pd
import traceback
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


/home/jatin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def read_UTK(startpath):
    X = []
    y = []
    try:
        for root, dirs, files in os.walk(startpath):
            for file in files:
                if file[-3:] == 'jpg':
                    names = file.split('_')
                    if int(names[0]) > 17 and int(names[0]) < 31:
                        y.append(names[1])
                        img = cv2.imread(root + file,cv2.IMREAD_GRAYSCALE)
                        X.append(img)
                
    except Exception as err:
        traceback.print_tb(err.__traceback__)
    return np.array(X,dtype='float'), np.array(y,dtype='uint8')

In [6]:
UTK_path = '../Data/UTKFace/'
X,y = read_UTK(UTK_path)

In [7]:
print(X.shape)
print(y.shape)

(8428, 200, 200)
(8428,)


In [8]:
y_translation = {
    0: 'Male',
    1: 'Female',
}

In [9]:
X = np.expand_dims(X, -1)
oht = OneHotEncoder(categories='auto', sparse=False)
y = oht.fit_transform(y.reshape(-1,1))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [11]:
# IMG_SIZE = 200
# model = Sequential()
# model.add(Conv2D(64, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
# model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
# # model.add(MaxPooling2D(pool_size=(2,2)))
# # model.add(BatchNormalization())
# model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
# model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(BatchNormalization())
# # model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2,2)))
# # model.add(BatchNormalization())
# # model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2,2)))
# # model.add(BatchNormalization())
# model.add(Dropout(rate=0.8))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(2, activation = 'softmax'))

In [12]:
classes = 2
inputShape = (200,200,1)

In [13]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=inputShape, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(classes, activation = 'relu'))

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X, y, batch_size=100, epochs=5)

Epoch 1/5
4900/8428 [================>.............] - ETA: 294s - loss: 4.4084 - acc: 0.4660

In [19]:
np.unique(y,axis=0, return_counts=True)

(array([[0., 1.],
        [1., 0.]]), array([4902, 3526]))